In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
nl_1k = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_nolate_1k.csv")
nl_all = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_nolate_all.csv")
yl_1k = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_yeslate_1k.csv")
yl_all = pd.read_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_yeslate_all.csv")

In [29]:
df = pd.DataFrame()
df['username'] = nl_1k['username']
reviews = [nl_1k,nl_all,yl_1k,yl_all]

#get count for non zero rows 
cols = nl_1k.iloc[:,1:].columns
df['review_count'] = nl_all[cols].gt(0.1).sum(axis=1)
df['late_review_count'] = yl_all[cols].gt(0.1).sum(axis=1)
#Drop Reviewers which have not reviewed enough movies, threshold is 10% 
rows_to_drop = df[df['review_count']<(len(cols)/10)].index
df.drop(index= rows_to_drop, axis=0, inplace= True)

#Average the columns

#for x in reviews:
    #df[f"mean_{x}"] = x.iloc[:,1:].mean(axis)=1

df['mean_nl_1k'] = nl_1k.iloc[:,1:].mean(axis=1)
df['mean_nl_all'] = nl_all.iloc[:,1:].mean(axis=1)
df['mean_yl_1k'] = yl_1k.iloc[:,1:].mean(axis=1)
df['mean_yl_all'] = yl_all.iloc[:,1:].mean(axis=1)

#Rank the columns 
rank_df = pd.DataFrame()
for col in df.columns[3:8]:
    rank_df[col] = df[col].rank(ascending = False)
rank_df = rank_df.add_suffix('_rank')
#merge the ranks with the scores
frames = [df, rank_df]
merged_df = pd.concat(frames,axis=1)

#back testing algorithm
#Rerun the above for movies up to a certain amount
movies = ['anora', 'conclave', 'the-brutalist', 'emilia-perez', 'nickel-boys', 'wicked-2024', 'maria-2024', 'oppenheimer-2023', 'poor-things-2023', 'the-holdovers', 'american-fiction', 'everything-everywhere-all-at-once', 'all-quiet-on-the-western-front-2022', 'the-fabelmans', 'the-banshees-of-inisherin', 'women-talking', 'top-gun-maverick', 'elvis-2022', 'coda-2021', 'the-power-of-the-dog', 'west-side-story-2021', 'dont-look-up-2021', 'king-richard', 'tick-tick-boom-2021', 'dune-2021', 'mank', 'nomadland', 'the-trial-of-the-chicago-7', 'borat-subsequent-moviefilm', 'promising-young-woman', 'palm-springs-2020', 'parasite-2019', '1917', 'once-upon-a-time-in-hollywood', 'jojo-rabbit', 'green-book', 'roma-2018', 'black-panther', 'bohemian-rhapsody', 'eighth-grade', 'can-you-ever-forgive-me', 'the-favourite', 'cold-war-2018', 'the-shape-of-water', 'three-billboards-outside-ebbing-missouri', 'lady-bird', 'get-out-2017', 'call-me-by-your-name', 'dunkirk-2017', 'i-tonya', 'blade-runner-2049', 'moonlight-2016', 'la-la-land', 'hidden-figures', 'arrival-2016', 'lion', 'spotlight', 'the-big-short', 'the-revenant-2015', 'the-martian', 'mad-max-fury-road', 'birdman-or-the-unexpected-virtue-of-ignorance', 'boyhood', 'the-grand-budapest-hotel', 'the-imitation-game']
#reverse the list
movies = movies[::-1]
print(len(movies))

#Start with the first 5 years and look up to the first 9 years
#for i in 



merged_df.to_csv("/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/modified_data/connor_modified_data/z_scores/zscore_collection.csv", index = False)




65
